In [1]:
#Q1
# Function to calculate the after-tax present value of the vaccine project
def calculate_npv(initial_investment, sales_per_unit, units_sold_annually, wage_rate_per_hour, 
                  hours_of_labor_annually, cost_per_ton_raw_material, tons_of_raw_material_annually, 
                  inflation_rate, corporate_tax_rate, depreciation, interest_rate, years):
    # Initialize list to hold the cash flows
    cash_flows = []

    # Calculate the cash flows for each year
    for year in range(1, years + 1):
        # Adjust for inflation
        sales = sales_per_unit * (1 + inflation_rate) ** (year - 1) * units_sold_annually
        labor_cost = wage_rate_per_hour * (1 + inflation_rate) ** (year - 1) * hours_of_labor_annually
        material_cost = cost_per_ton_raw_material * (1 + inflation_rate) ** (year - 1) * tons_of_raw_material_annually
        operating_income = sales - (labor_cost + material_cost)
        profit_before_tax = operating_income - depreciation
        tax = profit_before_tax * corporate_tax_rate
        profit_after_tax = profit_before_tax - tax
        cash_flow = profit_after_tax + depreciation # Add back depreciation as it's a non-cash charge
        cash_flows.append(cash_flow)
    
    # Calculate the present value of the cash flows
    npv = -initial_investment
    for i, cash_flow in enumerate(cash_flows, start=1):
        npv += cash_flow / (1 + interest_rate) ** i
    
    # Return the NPV rounded to the nearest dollar
    return round(npv)

# Define the parameters based on the provided scenario
initial_investment = 10000000
sales_per_unit = 3.30
units_sold_annually = 1000000
wage_rate_per_hour = 30
hours_of_labor_annually = 10000
cost_per_ton_raw_material = 100
tons_of_raw_material_annually = 100
inflation_rate = 0.10  # 10% inflation rate
corporate_tax_rate = 0.34
depreciation = initial_investment / 5
interest_rate = 0.05
years = 5

# Calculate the NPV
npv = calculate_npv(initial_investment, sales_per_unit, units_sold_annually, wage_rate_per_hour, 
                    hours_of_labor_annually, cost_per_ton_raw_material, tons_of_raw_material_annually, 
                    inflation_rate, corporate_tax_rate, depreciation, interest_rate, years)

print(f"The (after-tax) present value of the new vaccine project is: ${npv}")


The (after-tax) present value of the new vaccine project is: $3279795


In [6]:
import numpy as np

# Costs
c = 30 * 10000 * (1 + 0.1) ** np.arange(5) + 100 * 100 * (1 + 0.1) ** np.arange(5)

# Revenue
r = 3.3 * 1000000 * (1 + 0.1) ** np.arange(5)

# Profit
p = r - c

# Tax calculation
tax = (p - 10e6 / 5 * np.ones(5)) * 0.34

# Profit after tax
p1 = p - tax

# Discounting future cash flows to present value
discounted_cash_flows = np.array([-10e6] + list(p1)) / (1.05) ** np.arange(6)
#revise 1.03->1.05

# Rounding the sum of discounted cash flows
result = round(np.sum(discounted_cash_flows))
print("Result:", result)


Result: 3279795


In [1]:
#Q2
2*80-100

60

In [7]:
#Q2
import numpy as np

# Given values
P1 = 80
P2 = 100
C1 = 0.03
C2 = 0.06
F = 100
T = 10

# Solving the system of linear equations
X = np.linalg.solve([[F, F], [C1, C2]], [F, 0])

# Calculating the price P
P = np.dot(X, [P1, P2])

# Calculating the yield to maturity S
S = (F / P) ** (1 / T) - 1

print("X:", X)
print("P:", P)
print("S:", S)

X: [ 2. -1.]
P: 60.0
S: 0.05240977914892553


In [4]:
#Q3
import math
#P = mortegage value n1 = original terms, n2= rest terms, r1 = ir1, r2 =ir2
def ext_term(P, n1, m, r1, r2, n2): 
    A1 = r1/m*P*(1-1/(1+r1/m)**(n1*m))**(-1)
    P1 = A1/(r1/m)*(1-1/(1+r1/m)**((n1-n2)*m))
    a = P1 * r2/m / A1
    b = 1 - a
    t = -math.log(b, (1+r2/m))
    return t - (n1-n2)*m

ext_term(P=1000000, n1=30, m=12, r1=0.05, r2=0.06, n2=5)

87.5845978486002

In [3]:
#Q4
def cash_flow_present_value(cash_flow, effective_rate):
    #"""Enter effective rate in percentage"""
    if isinstance(effective_rate, list) or isinstance(effective_rate, tuple):
        spr = [1] + [1 + r/100 for r in effective_rate]
        p = 0.
        for i, ai in enumerate(cash_flow):
            p += ai / (spr[i]) ** i
        return p
    r = 1 + effective_rate / 100
    res = 0
    for i, ai in enumerate(cash_flow):
        res += ai / r ** i
    return res

class Bond:
    #"""Implementation of a bond. A face value $30000, 9% bond that matures in 30 periods (years) that pays its coupons twice a period (year) should be entered as Bond(9, 30, 2, 300)"""
    def __init__(self, coupon_percentage, maturity_years, coupon_freq, face_value):
        self.percentage = coupon_percentage / 100
        self.maturity_years = maturity_years
        self.coupon_freq = coupon_freq
        self.face_value = face_value
        flow = [0] + [coupon_percentage / coupon_freq] * maturity_years * coupon_freq
        flow[-1] += face_value
        self.flow = tuple(flow)
    
    @property
    def coupon_payment(self):
        return self.percentage * self.face_value / self.coupon_freq

def QM_duration(cash_flow_or_bond, rates, compound_freq):
    #"""Enter a list of spot rates (in percentage) to calculate the quasi-modified duration.
    #Keep in mind if the price at rates = sk + y is P(y), then P'(0) = -Dq * P(0)

    #This trims the first term to keep consistent with the bond calculations. i.e. following the book. calculating the
    #quasimodified duration for (x0, x1, ..., xn) where x0 is the flow on year 0
    
    #You can actually feed a bond into this function and we will handle it for you"""
    
    if isinstance(cash_flow_or_bond, Bond):
        cash_flow = cash_flow_or_bond.flow
    else:
        cash_flow = cash_flow_or_bond
    
    assert len(cash_flow) <= len(rates) + 1, f"Wrong number of periods: we have {len(rates)} years of spot rates but {len(cash_flow)} years bond"

    # First calculate the pv of cash flow
    spr = [1] + [1 + r/100 for r in rates]
    res = 0
    for i, ai in enumerate(cash_flow):
        res += ai / (spr[i]) ** i
    PV = res

    x = cash_flow[1:]
    s = [1 + r/100 for r in rates]
    m = compound_freq
    n = len(cash_flow) - 1
    res = 0.
    for _k in range(n):
        k = _k + 1
        rs = (k/m) * x[_k] * (s[_k]) ** (-k-1)
        res += rs
    return res/PV

def immunize_parallel(bond1: Bond, bond2: Bond, obligation_cash_flow, rates, verbose: bool = False):
    # Calculate the PVob, Dob of obligation
    # Calculate the PV1, D1 of bond1
    # Calculate the PV2, D2 of bond2

    obligation = obligation_cash_flow

    # First do error checking
    num_years_total = max(bond1.maturity_years, bond2.maturity_years, len(obligation) - 1)
    enough_rates = len(rates) >= num_years_total
    if not enough_rates:
        raise ValueError(f"Expected at least {num_years_total} years of rates, found {len(rates)}")

    PVob = cash_flow_present_value(obligation, rates)
    PV1 = cash_flow_present_value(bond1.flow, rates)
    PV2 = cash_flow_present_value(bond2.flow, rates)
    
    Dob = QM_duration(obligation, rates, 1)
    D1 = QM_duration(bond1, rates, 1)
    D2 = QM_duration(bond2, rates, 1)

    if verbose:
        print(f"Duration of bond 1: {D1}")
        print(f"Duration of bond 2: {D2}")
        print(f"Duration of obligation: {Dob}")
        print(f"Present value of bond 1: {PV1}")
        print(f"Present value of bond 2: {PV2}")
        print(f"Present value of obligation: {PVob}")

    # Solve for
    # PV1   PV2     x1 = PVob
    # PV1D1 PV2D2   x2 = PVob Dob

    # Hence
    # x1 =  |PV1   PV2  |-1  |PVob    |
    # x2 =  |PV1D1 PV2D2|    |PVob Dob|
    det = PV1 * PV2 * D2 - PV2 * PV1 * D1
    if abs(det) < 1e-5:
        raise ValueError("This portfolio cannot be immunized - the equations have no solutions")

    x1 = 1/det * (PV2 * PVob * D2 - PVob * Dob * PV2)
    x2 = 1/det * (PV1 * PVob * Dob - PVob * D1 * PV1)

    if verbose:
        print(f"Buy {x1} units of bond 1 and {x2} units of bond 2")

    return x1, x2


## The inputs: in percentage
spot_rates = [5.25, 5.50, 5.75, 6.00, 6.25]
bond1 = Bond(6, 5, 1, 100)
bond2 = Bond(10, 4, 1, 100)
obligation = (0, 50000, 51000 + 51000, 52000 + 52000, 53000 + 53000, 54000)
immunize_parallel(bond1, bond2, obligation, spot_rates)


(-2216.560382053845, 5003.967355938829)

In [9]:
import numpy as np

# Defining the spot rates
s = np.array([525, 550, 575, 600, 625]) / 1e4

# Calculations for the first set of cash flows
pv1 = np.array([6] * 4 + [106]) / (1 + s) ** np.arange(1, 6)
P1 = np.sum(pv1)
dP1dlambda = np.array([6] * 4 + [106]) * np.arange(1, 6) * (1 + s) ** -np.arange(2, 7)
D1 = np.sum(dP1dlambda) / P1

# Calculations for the second set of cash flows
pv2 = np.array([10] * 3 + [110]) / (1 + s[:4]) ** np.arange(1, 5)
P2 = np.sum(pv2)
dP2dlambda = np.array([10] * 3 + [110]) * np.arange(1, 5) * (1 + s[:4]) ** -np.arange(2, 6)
D2 = np.sum(dP2dlambda) / P2

# Calculations for the observed cash flows
ob = np.array([50000, 102000, 104000, 106000, 54000])
P_ob = np.sum(ob / (1 + s) ** np.arange(1, 6))
D_ob = np.sum(ob * np.arange(1, 6) * (1 + s) ** -np.arange(2, 7)) / P_ob

# Solving the linear equations
x = np.round(np.linalg.solve([[P1, P2], [P1 * D1, P2 * D2]], [P_ob, P_ob * D_ob]), 3)

print("x:", x)


x: [-2216.56   5003.967]


In [5]:
import numpy as np
from numpy import ndarray
class OptimizeStockPortfolio:
    def __init__(self, expected_returns: list[float], covariances: np.ndarray[np.float64]):
        #"""Solves the Markovitz problem. Refer to the example below"""
        r, w = covariances.shape
        self.n = len(expected_returns)

        if r != w or r != self.n:
            raise ValueError(f"Expected number of stocks equal number of correlations... got number of stocks: {self.n}, correlation array: {r}x{w}")

        if not np.all(covariances - covariances.T < 1e-6): #type: ignore
            raise ValueError(f"Expected the covariance to be a symmetric matrix")

        # if np.all(np.abs(covariances.diagonal() - 1) < 1e-6): #type: ignore
        #     print("All diagonal entries are 1, did you enter the correlation matrix?")

        self.expecteds = np.array(expected_returns)
        self.covariances = np.array(covariances)

    def returns(self, weights: ndarray[np.float64]) -> tuple[float,float]: #type: ignore
        r = weights.dot(self.expecteds)
        sd = weights.T @ self.covariances @ weights
        return r, sd #type: ignore

    def solve_analytical(self):
        n, = self.expecteds.shape
        A = np.array(self.covariances)
        b = np.ones((n,))

        A_inv: ndarray[np.float64]

        try:
            A_inv = np.linalg.inv(A)
        except LinAlgError:
            raise ValueError("The matrix is not invertible. Try solving it the numerical way")

        x = A_inv @ b
        best_weight = x[:n] / np.sum(x[:n])
        best_r, best_sd = self.returns(best_weight)

        print(f"Best average return: {best_r}")
        print(f"Minimum standard deviation: {best_sd}")
        print(f"Best weights: {best_weight}")

        return best_weight

    def solve_analytical_target_return(self, target_mean_return: float):
       # """Solves the Markowitz problem analytically. Enter the target mean return r-bar in percentage"""
        r_bar = target_mean_return
        n, = self.expecteds.shape

        # There are n + 2 variables in total, the equations (in compact form) are:
        # w1 + ... + wn = 1
        # w1r1 + w2r2 + ... + wnrn = r_bar
        # C w - l r - m = 0
        # where the n + 2 variables are w1 ... wn and l and m

        # So in the spirit of solving system of linear equations, we make the matrix A and vector b so that we solve Ax = b

        A = np.zeros((n+2, n+2))
        b = np.zeros((n+2,))

        # First equation
        A[0, :n] = 1
        b[0] = 1

        # Second equation
        A[1, :n] = self.expecteds
        b[1] = r_bar

        # Covariance equations
        A[2:, :n] = self.covariances
        A[2:, n] = -self.expecteds
        A[2:, n+1] = -1

        # Solve equation the lazy way:
        A_inv: np.ndarray[np.float64]

        try:
            A_inv = np.linalg.inv(A)
        except LinAlgError:
            raise ValueError("The matrix is not invertible. Try solving it the numerical way")

        x = A_inv @ b
        best_weight = x[:n]
        best_r, best_sd = self.returns(best_weight)

        print(f"Best average return: {best_r}")
        print(f"Minimum standard deviation: {best_sd}")
        print(f"Best weights: {best_weight}")

        return best_weight

    def solve_tan_analytical(self, risk_free_rate: float):
        n, = self.expecteds.shape
        A = np.array(self.covariances)
        b = self.expecteds - risk_free_rate

        A_inv: ndarray[np.float64]

        try:
            A_inv = np.linalg.inv(A)
        except LinAlgError:
            raise ValueError("The matrix is not invertible. Try solving it the numerical way")

        x = A_inv @ b
        best_weight = x[:n] / np.sum(x[:n])
        best_r, best_sd = self.returns(best_weight)

        print(f"Best average return: {best_r}")
        print(f"Minimum standard deviation: {best_sd}")
        print(f"Best weights: {best_weight}")

        return best_weight

def markowitz_analytical(expected_returns, covariances):
    #"""Solves the classical markowitz problem by using approximation approach"""
    s = OptimizeStockPortfolio(expected_returns, covariances)
    return s.solve_analytical()

def markowitz_riskfree_analytical(expected_returns, covariances, risk_free):
    #"""Solves markowitz problem with risk free rate. Enter risk free rate in percentage"""
    s = OptimizeStockPortfolio(expected_returns, covariances)
    return s.solve_tan_analytical(risk_free)

def markowitz_fixed_mean_analytical(expected_returns, covariances, target_mean_return):
    #"""Solves markowitz problem with a target mean return. Enter risk free rate in percentage"""
    s = OptimizeStockPortfolio(expected_returns, covariances)
    return s.solve_analytical_target_return(target_mean_return)

In [8]:
# Q5,6
returns = np.array([69, 65, 44, 36, 41])

covs = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])

print("Q9 - aim for 85% return")
w = markowitz_fixed_mean_analytical(returns, covs, 85)
print(w[0] * 10000)

print("\n\nQ10")
markowitz_analytical(returns, covs)

print("\n\nQ11")
w = markowitz_riskfree_analytical(returns, covs, 8)
print(w[0])

Q9 - aim for 85% return
Best average return: 84.99999999999994
Minimum standard deviation: 0.17731040848882412
Best weights: [ 0.86487964  0.96102418 -0.68060261  0.24788048 -0.3931817 ]
8648.796400853185


Q10
Best average return: 53.00171526586621
Minimum standard deviation: 0.12381360777587187
Best weights: [ 0.0971984   0.53230417 -0.15208691  0.60777587 -0.08519154]


Q11
Best average return: 105.6597804543375
Minimum standard deviation: 0.2686921972021493
Best weights: [ 1.36053514  1.23782843 -1.02184022  0.01551304 -0.59203639]
1.3605351425522179


In [1]:
#Q5
import numpy as np

# Covariance matrix V
V = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])

# Expected returns of assets
r_bar = np.array([0.69, 0.65, 0.44, 0.36, 0.41])

# Constructing the augmented matrix for the linear system
A = np.vstack([
    np.hstack([V, -r_bar[:, np.newaxis], -np.ones((5, 1))]),
    np.hstack([r_bar, [0], [0]]),
    np.hstack([np.ones(5), [0], [0]])
])

# Constructing the right-hand side of the equation
b = np.append(np.zeros(5), [0.85, 1])

# Solving the linear system
solution = np.linalg.solve(A, b)

# Extracting the weights and normalizing them to sum to 1
weights = solution[:5] / sum(solution[:5])

weights


array([ 0.86487964,  0.96102418, -0.68060261,  0.24788048, -0.3931817 ])

In [2]:
#Q6
#use w_1
import numpy as np

# Define the covariance matrix V and expected returns r_bar
V = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])
r_bar = np.array([0.69, 0.65, 0.44, 0.36, 0.41])
r_f = 0.08

# Solve for weights of one fund
v_1 = np.linalg.solve(V, r_bar - r_f)
w_1 = v_1 / sum(v_1)

# Mean return
MR1 = r_bar.T @ w_1
Var1 = w_1.T @ V @ w_1

# Desired mean
Desired_mean = 0.8

# Find weights of portfolio with desired mean
alpha = (Desired_mean - MR1) / (r_f - MR1)
w_2 = (1 - alpha) * w_1

# Variance of portfolio (for one fund s.d. = (1 - alpha) s.d. of one fund)
Var = (1 - alpha)**2 * w_1.T @ V @ w_1
Var2 = w_2.T @ V @ w_2

w_1, MR1, Var1, w_2, MR1, Var, Var2


(array([ 1.36053514,  1.23782843, -1.02184022,  0.01551304, -0.59203639]),
 1.0565978045433755,
 0.26869219720214976,
 array([ 1.00305909,  0.91259315, -0.75335512,  0.01143704, -0.43648081]),
 1.0565978045433755,
 0.14604562794034198,
 0.146045627940342)

In [5]:
#Q7
V = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])
r_bar = np.array([0.69, 0.65, 0.44, 0.36, 0.41]).reshape(-1, 1)
var_v = 0.04
beta = np.array([1.16, 0.70, 1.15, 0.98, 1.01]).reshape(-1, 1)
cov_v = 2*beta * var_v

# Constructing the matrices for the linear system
A = np.block([
    [2 * V, -np.ones((5, 1)), -r_bar],
    [np.ones((1, 5)), 0, 0],
    [r_bar.T, 0, 0]
])
b = np.vstack((2 * cov_v, 1, 0.2))

# Solving the linear system
OPW_13 = np.linalg.solve(A, b)
OPW_13


array([[-0.46450001],
       [-0.2326239 ],
       [ 0.61892922],
       [ 0.85356276],
       [ 0.22463193],
       [ 0.2777491 ],
       [-0.30438314]])

In [9]:
#Q8
def investment_value_after_mortgage(P, r_mortgage, r_investment, term_years):
    m = 12  # monthly compounding and payments
    term_months = term_years * m

    # Monthly mortgage payment calculation
    monthly_payment = r_mortgage / m * P / (1 - (1 + r_mortgage / m) ** -term_months)

    # Initial investment amount
    investment_value = P  # same as the mortgage amount

    for _ in range(term_months):
        # Increase investment by monthly growth rate
        investment_value *= (1 + r_investment / m)
        
        # Subtract monthly mortgage payment
        investment_value -= monthly_payment

    return round(investment_value)

# Mortgage and investment details
P = 500000  # Principal amount of the mortgage
r_mortgage = 0.06  # Annual interest rate of the mortgage
r_investment = 0.01 * 12  # Monthly growth rate of the S&P 500 investment, converted to annual rate
term_years = 30  # Term of the mortgage in years

investment_value_after_mortgage(P, r_mortgage, r_investment, term_years)


7497783

In [14]:
#Q9
# constant-proportion rebalancing FPQ17

def Cons_propo_rebal(prop_of_cash,prop_of_stock,days,initial):
    stock = prop_of_stock * initial
    cash = prop_of_cash * initial
    for i in range(days):
        Ex = ((stock)/2 + 2*(stock))/2
        stock = prop_of_stock*(Ex+cash)
        cash = prop_of_cash*(Ex+cash)
    E = stock + cash
    print(E)
    return E

Cons_propo_rebal(0.5,0.5,20,100)

1054.50938424492


1054.50938424492

In [18]:
#Q10
import cvxpy as cp

# Given data
V = np.array([ # covariance matrix
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])
r_bar = np.array([0.69, 0.65, 0.44, 0.36, 0.41]) # expected returns

# The variable we are solving for
weights = cp.Variable(5)

# The objective is to minimize the portfolio variance
# and maximize the expected return
risk = cp.quad_form(weights, V)
return_portfolio = r_bar @ weights
objective = cp.Minimize(risk - return_portfolio)

# The sum of the weights must be 1
constraints = [cp.sum(weights) == 1]

# Solving the problem
prob = cp.Problem(objective, constraints)
prob.solve()

# The optimal weights
optimal_weights = weights.value

# Output the optimal weights
print("The optimal weights for the portfolio are:")
print(optimal_weights)

# Specifically, to output the weight for Meta Platforms, Inc. (formerly Facebook, ticker META):
meta_weight = optimal_weights[4]  # Assuming the 5th stock in the list is Meta/Facebook
print(f"The optimal weight for Meta Platforms, Inc. (META) is: {meta_weight:.2f}")


The optimal weights for the portfolio are:
[ 2.39308176  1.81446541 -1.7327044  -0.46855346 -1.00628931]
The optimal weight for Meta Platforms, Inc. (META) is: -1.01


In [1]:
#Q11 不一定对
# Given data
initial_investment = 200  # SVB's investment in the bond in billion dollars
coupon_rate = 6  # Annual coupon rate in percent
face_value_bond = 100  # Standard face value of the bond (assumed to be 100)
withdrawal_requests = 180  # Clients' withdrawal requests in billion dollars

# New spot rates (after one year) in percent
spot_rates = {1: 11.25, 2: 11.50, 3: 11.75, 4: 12.00, 5: 12.25}

# The bond has 4 years and 4 coupon payments remaining
# Annual coupon payment (in billion dollars)
annual_coupon_payment = (coupon_rate / 100) * initial_investment

# Present value of future cash flows from the bond
# PV = C/(1+r1) + C/(1+r2)^2 + C/(1+r3)^3 + C/(1+r4)^4 + (F+C)/(1+r4)^4
# C is the annual coupon payment and F is the face value of the bond
# The face value is scaled by the initial investment amount
present_value_bond = sum([annual_coupon_payment / ((1 + spot_rates[year]/100) ** year) for year in range(1, 5)])
present_value_bond += (annual_coupon_payment + (face_value_bond / 100 * initial_investment)) / ((1 + spot_rates[4]/100) ** 4)

# Total cash after receiving the first coupon payment and selling the bond
total_cash = annual_coupon_payment + present_value_bond

# Calculating the shortfall and the required bailout amount
shortfall = withdrawal_requests - total_cash
shortfall_billion = round(shortfall)  # rounding to nearest integer

shortfall_billion, present_value_bond, total_cash


(-3, 171.3936978629236, 183.3936978629236)

In [3]:
# Given data
initial_investment = 200  # SVB's investment in the bond in billion dollars
coupon_rate = 6  # Annual coupon rate in percent
withdrawal_requests = 180  # Clients' withdrawal requests in billion dollars

# Spot rates (after one year) in percent
spot_rates = {1: 11.25, 2: 11.50, 3: 11.75, 4: 12.00, 5: 12.25}

# The bond has 4 years and 4 coupon payments remaining
# Annual coupon payment (in billion dollars)
annual_coupon_payment = (coupon_rate / 100) * initial_investment

# Calculating the present value of future cash flows from the bond
# The present value of the bond should include 4 years of coupon payments and the face value at the end
present_value_bond_revised = sum([annual_coupon_payment / ((1 + spot_rates[year]/100) ** year) for year in range(1, 5)])
present_value_bond_revised += initial_investment / ((1 + spot_rates[4]/100) ** 4)

# Total cash after receiving the first coupon payment and selling the bond
total_cash_revised = annual_coupon_payment + present_value_bond_revised

# Calculating the shortfall and the required bailout amount
shortfall_revised = withdrawal_requests - total_cash_revised
shortfall_billion_revised = round(shortfall_revised)  # rounding to nearest integer

# Output the revised results
shortfall_billion_revised, present_value_bond_revised, total_cash_revised



(4, 163.76748092206566, 175.76748092206566)

In [6]:
# To calculate the bailout money needed by SVB, we first need to calculate the value of the U.S. Treasury bond
# that SVB holds after the first coupon payment and its sale.

# Given data:
initial_investment = 200  # in billion dollars
annual_coupon_rate = 0.06  # 6%
coupon_payment = annual_coupon_rate * initial_investment  # Coupon payment received on March 1, 2023
years_remaining = 4  # 4 years remaining after the first coupon
withdrawal_request = 180  # in billion dollars

# New spot rate curve (after +6% increase across all maturities):
spot_rates = {
    1: 0.1125,  # 11.25%
    2: 0.1150,  # 11.50%
    3: 0.1175,  # 11.75%
    4: 0.1200   # 12.00%
}

# Calculate the present value of the remaining cash flows of the bond
present_value_bond = 0
for year in range(1, years_remaining + 1):
    cash_flow = coupon_payment if year < years_remaining else coupon_payment + initial_investment
    discount_factor = 1 / (1 + spot_rates[year]) ** year
    present_value_bond += cash_flow * discount_factor

# Calculate the total cash SVB has after selling the bond and receiving the coupon payment
total_cash = present_value_bond + coupon_payment

# Calculate the amount of bailout money needed
bailout_needed = withdrawal_request - total_cash
bailout_needed_billion = round(bailout_needed)

bailout_needed_billion


4

In [1]:
#Q12
import numpy as np

def IRRPoly(cash_flow, m):
    coefficients = np.flip(cash_flow)
    roots = np.roots(coefficients)
    irr_poly = np.real(roots[(roots >= 0)  & (np.iscomplex(roots) == False)])[0]
    irr = m*(1 - irr_poly) / irr_poly
    return irr

# Bond characteristics
face_value = 100
coupon_payment = 4  #yearly
payments_per_year = 2
years_to_maturity = 5
current_price = 110

# Cash flows for the bond

cash_flows = (coupon_payment / payments_per_year)*np.ones(years_to_maturity * payments_per_year)
cash_flows[-1] += face_value
print(cash_flows)
cash_flows = np.insert(cash_flows, 0, -current_price)
print(cash_flows)

# Find YTM using IRRPoly function
ytm = IRRPoly( cash_flows, payments_per_year)

# Convert to a percentage and display the result
ytm_percentage = ytm * 100
print(f"The bond's yield to maturity is {ytm_percentage:.2f}%")

def bond_price(F, C, lamb, m, n):
    '''
    F is the face value of the bond
    C is the yearly coupon payment
    lamb is the yield of the bond 
    m is the number of coupon payments per year
    n is the number of coupon payments remaning
    '''
    r = lamb / m  # Semi-annual yield
    discount_factor = (1 + r) ** n
    bond_price = (C / r) * (1 - 1 / discount_factor) + F / discount_factor
    return bond_price

face_value = 100
coupon_rate = 0.1
periods=10
frequency = 1
yield_to_maturities = ytm_percentage
coupon_payment_per_period = face_value * coupon_rate / frequency
bond_price(face_value, coupon_payment_per_period, ytm, frequency, periods * frequency)

[  2.   2.   2.   2.   2.   2.   2.   2.   2. 102.]
[-110.    2.    2.    2.    2.    2.    2.    2.    2.    2.  102.]
The bond's yield to maturity is 1.89%


173.21395912251086

In [ ]:
#11

In [18]:
D=0
for i in range(1,17):
    D+=(-1/1.07)*((i*0.08)/((1+0.035)**(i)))
D+(-1/1.07)*(8/(1+0.07)**8)

-11.379755780988436

In [1]:
B=0
for i in range(1,17):
    B+=4*(1/((1.035)**i))
B+(100/((1.035)**16))

106.04705840407456

In [25]:
D=0
for i in range(1,17):
    D+=(1/106.047)*((i*4)/((1+0.035)**(i)))
print((D+((1600/((1+0.035)**16)))/106.047)/2)
print((D+((800/((1+0.07)**16)))/106.047)) 

6.123429947101408
6.101077547635899
